In [ ]:
from dask_gateway import GatewayCluster
import dask.distributed
import dask.utils
import dask.array
import dask
import planetary_computer
from pystac_client import Client
import odc.stac
import numpy
import xarray
import rasterio
import rasterio.enums
import gc
import math
import os
from scipy import ndimage

In [ ]:
def apply_sen2_vld_msk(scns_xa, bands, qa_pxl_msk="SCL", out_no_data_val=0):
    scns_lcl_xa = scns_xa.copy()
    for band in bands:
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 0] = out_no_data_val  # No Data
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 1] = out_no_data_val  # Saturation
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 2] = out_no_data_val  # Cast Shadow
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 3] = out_no_data_val  # Cloud Shadows
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 8] = out_no_data_val  # Cloud Medium Probability
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 9] = out_no_data_val  # Cloud High Probability
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 10] = out_no_data_val # Thin Cirrus
    return scns_lcl_xa

def get_img_metadata(img_file):
    img_data_obj = rasterio.open(img_file)
    img_bounds = img_data_obj.bounds
    img_bbox = [img_bounds.left, img_bounds.bottom, img_bounds.right, img_bounds.top]
    img_x_res, img_y_res  = img_data_obj.res
    if img_y_res > 0:
        img_y_res = img_y_res * (-1)
    img_data_obj = None
    return img_bbox, img_x_res, img_y_res

def get_img_band_array(img_file, band=1):
    img_data_obj = rasterio.open(img_file)
    img_arr = img_data_obj.read(band)
    img_data_obj = None
    return img_arr

In [ ]:
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.
cluster.adapt(minimum=4, maximum=24)
print(cluster.dashboard_link)

client = dask.distributed.Client(cluster, timeout=10)
odc.stac.configure_rio(cloud_defaults=True, client=client)

In [ ]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [ ]:
# Date range of the ROI
time_range = "2018-01-01/2018-12-31"
date_str = "201812"
# Bands to be read
bands = ["B03", "B04", "B08", "B11", "SCL"]

In [ ]:
# Create a 5x5 circular operator
morph_op3 = ndimage.generate_binary_structure(2, 1)
morph_op5 = numpy.zeros((5,5))
morph_op5[2,2] = 1
morph_op5 = ndimage.binary_dilation(morph_op5, structure=morph_op3, iterations=2).astype(morph_op5.dtype)

In [ ]:
# Define the tiles to be processed.

# All of Africa Tiles:
tiles = ["S10E039", "S25E044", "S13E049", "S10E040", "N05E007", "N22E039", "N05E008", "N22E036", "S28E032", "N29E034", "N21E037", "S25E032", "N27E035", "N11W016", "N12W017", "N14E042", "S16E045", "S18E044", "S10E047", "S09E013", "S07E012", "S18E035", "S13E047", "S30E030", "N17E042", "S16E044", "S15E050", "S15E045", "N14E043", "S07E013", "N06E000", "N09W014", "S02E010", "N27E034", "S26E032", "N24E035", "N06E008", "N05E009", "N12W016", "S03E039", "S13E040", "N07W012", "N21E039", "N09W013", "N05E006", "S03E040", "N27E033", "S13E048", "N21E040", "N19E037", "N06E005", "N24E038", "N11E045", "S08E039", "N12E051", "S06E038", "S15E040", "S05E039", "S16E049", "S20E044", "S01E009", "S16E038", "S20E035", "S18E036", "S15E047", "S16E046", "N17E040", "N19E041", "N12E043", "N11W015", "S20E043", "N17W017", "N14W016", "S28E031", "N19E038", "N03E009", "N17E039", "N00E008", "N27E036", "N14W017", "N12W015", "S20E057", "N11W014", "S23E043", "S05E053", "N00E009", "S04E011", "S33E027", "N17E041", "S15E046", "S15E049", "S23E035", "S20E034", "S16E039", "S02E009", "S01E008", "N07W011", "N06E004", "N05E005", "S05E038", "S06E039", "S16E040", "N12E050", "N07E003", "N05W002", "N06W003", "S17E036", "S12E044", "N13E042", "S32E027", "N15E040", "N15W017", "S22E043", "N10W014", "N13W015", "N01E009", "N26E037", "S29E030", "S09E046", "N06W004", "N06W011", "N01E006", "N05W010", "N07E004", "S07E039", "N15W018", "S12E043", "S31E029", "S32E028", "S21E034", "S01E042", "S22E035", "S17E049", "S14E048", "S32E029", "S17E038", "S04E055", "S21E035", "N07E005", "N06W010", "N06W005", "N25E038", "S09E039", "S11E043", "S04E039", "S14E040", "S21E043", "N10W015", "S10E013", "N16W017", "N18E038", "S09E047", "N00E042", "S29E031", "N23E035", "N26E036", "N25E037", "N16E039", "N02E009", "S17E037", "N06W002", "N05W003", "N07E002", "N16E040", "S14E047", "N18E041", "S12E045", "N13E043", "N25E035", "N05W009", "N26E034", "S27E032", "N04E009", "N28E035", "S22E040", "N13W016", "N13E054", "S31E030", "N16E042", "S17E044", "N08W014", "S19E034", "S05E012", "S03E010", "S02E040", "S01E041", "N20E040", "S12E048", "S11E049", "S17E043", "S12E040", "N13E053", "S19E057", "N20W017", "N05W006", "N23E038", "N08W013", "S11E040", "N08W012", "N23E039", "N05W007", "N06W006", "N28E033", "S01E040", "S02E041", "S24E044", "S12E049", "N15E042", "S19E044", "N18E042", "N06W001", "S03E011", "S08E013", "S06E012", "S19E035", "N07E001", "N20E037", "N00E041", "N23E036", "N04E008", "N05W008", "N06W009", "N13W017"]
tiles = tiles[0:75]
# All 2022 Region Tiles
#tiles = ["S05W081", "N01W081", "N02W080", "S03W081", "N02W079", "S03W080", "N01W080", "S02W080", "S04W082", "N00W081", "S02W081", "N22W091", "N19W104", "N19W089", "N17W095", "N19W094", "N21W098", "N21W097", "N19W103", "N22W106", "N19W093", "N27W110", "N19W092", "N21W106", "N22W098", "N19W095", "N17W094", "N19W088", "N21W091", "N19W105", "N22W090", "N17W096", "N27W114", "N24W098", "N19W097", "N22W087", "N24W108", "N22W088", "N29W112", "N27W113", "N24W107", "N17W101", "N17W099", "N17W098", "N17W100", "N27W112", "N21W088", "N29W113", "N19W091", "N22W089", "N21W087", "N19W096", "N29W114", "N27W098", "N26W112", "N25W113", "N20W105", "N20W088", "N28W113", "N18W101", "N16W098", "N28W114", "N26W098", "N16W097", "N25W109", "N20W092", "N25W108", "N26W109", "N16W096", "N28W112", "N25W112", "N26W113", "N30W113", "N20W096", "N18W102", "N16W093", "N20W106", "N28W110", "N23W107", "N30W114", "N23W090", "N20W091", "N23W098", "N16W094", "N23W099", "N28W111", "N23W106", "N25W111", "N15W093", "N26W110", "N18W103", "N20W097", "N06W056", "N06W057", "N06W058", "N05W053", "N06W055", "N06W054", "N05W052", "N06W053", "N04W052", "N07W057", "N04W051", "N07W058", "N07W056", "N08W060", "N09W060", "N08W059", "N07W059", "N23E091", "N18E094", "N16E095", "N10E098", "N13E098", "N16E094", "N13E097", "N23E090", "N18E097", "N15E097", "N20E093", "N23E087", "N23E088", "N20E094", "N15E098", "N23E089", "N16E097", "N22E089", "N17E097", "N21E092", "N22E087", "N22E092", "N21E093", "N17E096", "N14E098", "N22E088", "N12E098", "N22E090", "N17E094", "N19E094", "N11E097", "N17E095", "N22E091", "N19E093", "N11E098", "N02E120", "N01E121", "S05E123", "N04E125", "S03E121", "S04E119", "S03E120", "S05E122", "N01E120", "N02E121", "N01E122", "N00E119", "S05E120", "S03E122", "N02E124", "S02E119", "S03E123", "N02E125", "N01E124", "S05E121", "S06E120", "S01E119", "S02E118", "N01E123", "N02E122", "S01E122", "S02E123", "N03E125", "S04E121", "S03E118", "N00E123", "N00E122", "N01E119", "S04E120", "S02E122", "S01E123", "S03E119", "S05E119", "S01E121", "S02E120", "S04E122", "N00E120", "S05E117", "N00E121", "S05E118", "S04E123", "S02E121", "S01E120", "N06E118", "S01E109", "N06E117", "S02E112", "N08E116", "S02E113", "N08E117", "N05E117", "N06E116", "N00E109", "S04E116", "N03E112", "S01E108", "N05E118", "N06E119", "S02E116", "S04E114", "N03E118", "N05E115", "S01E110", "N00E116", "N03E117", "S02E111", "S02E110", "N00E117", "N05E114", "N06E115", "S04E115", "S01E116", "N03E111", "N01E117", "S03E110", "N04E113", "N02E111", "N01E118", "N02E118", "S03E116", "N02E110", "N02E117", "S03E111", "N02E108", "N01E109", "S03E113", "N07E116", "N04E117", "S03E114", "S03E115", "N07E118", "N07E117", "S03E112", "N01E108", "N02E109", "S10E039", "S25E044", "S13E049", "S10E040", "N05E007", "N05E008", "S28E032", "S25E032", "N11W016", "N12W017", "S16E045", "S18E044", "S10E047", "S09E013", "S07E012", "S18E035", "S13E047", "S30E030", "S16E044", "S15E050", "S15E045", "S07E013", "N06E000", "N09W014", "S02E010", "S26E032", "N06E008", "N05E009", "N12W016", "S03E039", "S13E040", "N07W012", "N09W013", "N05E006", "S03E040", "S13E048", "N06E005", "S08E039", "S06E038", "S15E040", "S05E039", "S16E049", "S20E044", "S01E009", "S16E038", "S20E035", "S18E036", "S15E047", "S16E046", "N11W015", "S20E043", "N17W017", "N14W016", "S28E031", "N03E009", "N00E008", "N14W017", "N12W015", "N11W014", "S23E043", "N00E009", "S04E011", "S33E027", "S15E046", "S15E049", "S23E035", "S20E034", "S16E039", "S02E009", "S01E008", "N07W011", "N06E004", "N05E005", "S05E038", "S06E039", "S16E040", "N07E003", "N05W002", "N06W003", "S17E036", "S12E044", "S32E027", "N15W017", "S22E043", "N10W014", "N13W015", "N01E009", "S29E030", "S09E046", "N06W004", "N06W011", "N01E006", "N05W010", "N07E004", "S07E039", "N15W018", "S12E043", "S31E029", "S32E028", "S21E034", "S01E042", "S22E035", "S17E049", "S14E048", "S32E029", "S17E038", "S21E035", "N07E005", "N06W010", "N06W005", "S09E039", "S11E043", "S04E039", "S14E040", "S21E043", "N10W015", "S10E013", "N16W017", "S09E047", "N00E042", "S29E031", "N02E009", "S17E037", "N06W002", "N05W003", "N07E002", "S14E047", "S12E045", "N05W009", "S27E032", "N04E009", "S22E040", "N13W016", "S31E030", "S17E044", "N08W014", "S19E034", "S05E012", "S03E010", "S02E040", "S01E041", "S12E048", "S11E049", "S17E043", "S12E040", "N20W017", "N05W006", "N08W013", "S11E040", "N08W012", "N05W007", "N06W006", "S01E040", "S02E041", "S24E044", "S12E049", "S19E044", "N06W001", "S03E011", "S08E013", "S06E012", "S19E035", "N07E001", "N00E041", "N04E008", "N05W008", "N06W009", "N13W017", "N22E039", "N22E036", "N29E034", "N21E037", "N27E035", "N14E042", "N17E042", "N14E043", "N27E034", "N24E035", "N21E039", "N27E033", "N21E040", "N19E037", "N24E038", "N11E045", "N12E051", "N17E040", "N19E041", "N12E043", "N19E038", "N17E039", "N27E036", "S20E057", "S05E053", "N17E041", "N12E050", "N13E042", "N15E040", "N26E037", "S04E055", "N25E038", "N18E038", "N23E035", "N26E036", "N25E037", "N16E039", "N16E040", "N18E041", "N13E043", "N25E035", "N26E034", "N28E035", "N13E054", "N16E042", "N20E040", "N13E053", "S19E057", "N23E038", "N23E039", "N28E033", "N15E042", "N18E042", "N20E037", "N23E036", "S19E178", "S17E179", "S18W180", "S17E176", "S17E177", "S16W180", "S18W179", "S17E178", "S19E179", "S17W180", "S16E177", "S16E178", "S18E179", "S18E178", "S17W179", "S16E179", "S18E177"]
#tiles.reverse()

############## IGNORE - DEV ######################
## Test Tiles:
##tiles = ["N15W017", "N14W017", "N14W016", "N13W017", "N13W016", "N13W015", "N12W017", "N12W016", "N12W015", "S10E039"]
## Old Africa List...
##tiles = ['S10E039', 'S25E044', 'S13E049', 'S10E040', 'N05E007', 'N05E008', 'S28E032', 'S25E032', 'N11W016', 'N12W017', 'S16E045', 'S18E044', 'S10E047', 'S09E013', 'S07E012', 'S18E035', 'S13E047', 'S30E030', 'S16E044', 'S15E050', 'S15E045', 'S07E013', 'N06E000', 'N09W014', 'S02E010', 'S26E032', 'S19E044', 'N06E008', 'N05E009', 'N12W016', 'S03E039', 'S13E040', 'N07W012', 'N09W013', 'N05E006', 'S03E040', 'S13E048', 'N06E005', 'S08E039', 'S06E038', 'S15E040', 'S05E039', 'S16E049', 'S20E044', 'S01E009', 'S16E038', 'S20E035', 'S18E036', 'S15E047', 'S16E046', 'N11W015', 'S20E043', 'N17W017', 'N14W016', 'S28E031', 'N03E009', 'N00E008', 'N14W017', 'N12W015', 'S20E057', 'N11W014', 'S23E043', 'S05E053', 'N00E009', 'S04E011', 'S15E046', 'S15E049', 'S23E035', 'S20E034', 'S16E039', 'S02E009', 'S01E008', 'N07W011', 'N06E004', 'N05E005', 'S05E038', 'S06E039', 'S16E040', 'N07E003', 'N05W002', 'N06W003', 'S17E036', 'N15W017', 'S22E043', 'N10W014', 'N13W015', 'N01E009', 'S29E030', 'S09E046', 'N06W004', 'N06W011', 'N01E006', 'N05W010', 'N07E004', 'S07E039', 'N15W018', 'S12E043', 'S31E029', 'S32E028', 'S21E034', 'S01E042', 'S22E035', 'S17E049', 'S14E048', 'S32E029', 'S17E038', 'S04E055', 'S21E035', 'N07E005', 'N06W010', 'N06W005', 'S09E039', 'S11E043', 'S04E039', 'S14E040', 'S21E043', 'N10W015', 'S10E013', 'N16W017', 'S09E047', 'N00E042', 'S29E031', 'N02E009', 'S17E037', 'N06W002', 'N05W003', 'N07E002', 'S14E047', 'S12E045', 'N05W009', 'N04E009', 'S22E040', 'N13W016', 'S31E030', 'S17E044', 'N08W014', 'S19E034', 'S05E012', 'S03E010', 'S02E040', 'S01E041', 'S12E048', 'S11E049', 'S17E043', 'S12E040', 'S19E057', 'N20W017', 'N05W006', 'N08W013', 'S11E040', 'N08W012', 'N05W007', 'N06W006', 'S02E041', 'S24E044', 'S12E049', 'N06W001', 'S03E011', 'S08E013', 'S06E012', 'S19E035', 'N07E001', 'N00E041', 'N04E008', 'N05W008', 'N06W009', 'N13W017', 'S33E027', 'S12E044', 'S32E027', 'S27E032', 'S01E040']
## Borneo Tiles:
##tiles = ["N06E118", "S01E109", "N03E109", "N06E117", "S02E112", "N08E116", "S02E113", "N08E117", "N05E117", "N06E116", "N00E109", "N03E108", "S04E116", "N03E112", "S01E108", "N05E118", "N06E119", "S02E116", "S04E114", "N03E118", "N05E115", "S01E110", "N00E116", "N03E117", "S02E111", "S02E110", "N00E117", "N05E114", "N06E115", "S04E115", "S01E116", "N03E111", "N01E117", "S03E110", "N04E113", "N02E111", "N01E118", "N02E118", "S03E116", "N02E110", "N02E117", "S03E111", "N02E108", "N01E109", "S03E113", "N07E116", "N04E117", "S03E114", "S03E115", "N07E118", "N07E117", "S03E112", "N01E108", "N02E109"]
## Tiles for further mangroves areas
##tiles = ["S05W081", "N01W081", "N02W080", "S03W081", "N02W079", "S03W080", "N01W080", "S02W080", "S04W082", "N00W081", "S02W081", "N22W091", "N19W104", "N19W089", "N17W095", "N19W094", "N21W098", "N21W097", "N19W103", "N22W106", "N19W093", "N27W110", "N19W092", "N21W106", "N22W098", "N19W095", "N17W094", "N19W088", "N21W091", "N19W105", "N22W090", "N17W096", "N27W114", "N24W098", "N19W097", "N22W087", "N24W108", "N22W088", "N29W112", "N27W113", "N24W107", "N17W101", "N17W099", "N17W098", "N17W100", "N27W112", "N21W088", "N29W113", "N19W091", "N22W089", "N21W087", "N19W096", "N29W114", "N27W098", "N26W112", "N25W113", "N20W105", "N20W088", "N28W113", "N18W101", "N16W098", "N28W114", "N26W098", "N16W097", "N25W109", "N20W092", "N25W108", "N26W109", "N16W096", "N28W112", "N25W112", "N26W113", "N30W113", "N20W096", "N18W102", "N16W093", "N20W106", "N28W110", "N23W107", "N30W114", "N23W090", "N20W091", "N23W098", "N16W094", "N23W099", "N28W111", "N23W106", "N25W111", "N15W093", "N26W110", "N18W103", "N20W097", "N06W056", "N06W057", "N06W058", "N05W053", "N06W055", "N06W054", "N05W052", "N06W053", "N04W052", "N07W057", "N04W051", "N07W058", "N07W056", "N08W060", "N09W060", "N08W059", "N07W059", "N23E091", "N18E094", "N16E095", "N10E098", "N13E098", "N16E094", "N13E097", "N23E090", "N18E097", "N15E097", "N20E093", "N23E087", "N23E088", "N20E094", "N15E098", "N23E089", "N16E097", "N22E089", "N17E097", "N21E092", "N22E087", "N22E092", "N21E093", "N17E096", "N14E098", "N22E088", "N12E098", "N22E090", "N17E094", "N19E094", "N11E097", "N17E095", "N22E091", "N19E093", "N11E098", "N02E120", "N01E121", "S05E123", "N04E125", "S03E121", "S04E119", "S03E120", "S05E122", "N01E120", "N02E121", "N01E122", "N00E119", "S05E120", "S03E122", "N02E124", "S02E119", "S03E123", "N02E125", "N01E124", "S05E121", "S06E120", "S01E119", "S02E118", "N01E123", "N02E122", "S01E122", "S02E123", "N03E125", "S04E121", "S03E118", "N00E123", "N00E122", "N01E119", "S04E120", "S02E122", "S01E123", "S03E119", "S05E119", "S01E121", "S02E120", "S04E122", "N00E120", "S05E117", "N00E121", "S05E118", "S04E123", "S02E121", "S01E120"]
#################################################


n_tiles = len(tiles)

In [ ]:
n_tile = 0
# Iterate through the tiles.
for tile in tiles:
    print(f"{tile}: ({n_tile+1} of {n_tiles})")
    out_img_file = os.path.join("../gmw_2018_revised_ext_opt_s1", f'gmw_{tile}_2018_alerts_ext.tif')
    # Do not process tile if output file already exists.
    if not os.path.exists(out_img_file):
        # Define the GMW extent image file
        gmw_tile_img = f"../gmw_2018_ext_tiles/gmw_{tile}_2018_mng_ext.tif"
        # Get the bbox and image resolution of the input image.
        bbox, img_x_res, img_y_res = get_img_metadata(gmw_tile_img)
        
        # Search for scenes.
        search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=time_range, query={"eo:cloud_cover": {"lt": 50}})
        items = search.get_all_items()
        n_items = len(items)
        print(f"\tN Sen2 Scenes: {n_items}")
        
        ####################################################################
        # Find the scenes for the period of interest.
        s1_search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_range, query={"sar:polarizations": {"eq": ["VV", "VH"]}})
        s1_items = s1_search.get_all_items()
        n_s1_items = len(s1_items)
        print(f"\tN Sen1 Items = {n_s1_items}")
        s1_avail = True
        if n_s1_items < 1:
            s1_avail = False
        ####################################################################
        
        # Only continue analysis if there are scenes available.
        if n_items > 0:
            # Read the GMW extent into a numpy array
            gmw_ext_msk = get_img_band_array(gmw_tile_img)
            
            # Sign all the items
            signed_items = [planetary_computer.sign(item) for item in items]

            # Read the data into dask xarray structure
            sen2_scn_xa = odc.stac.stac_load(
                signed_items,
                bands=bands,
                groupby="solar_day",
                #dtype=numpy.uint16,
                chunks={"time":24, "latitude": 1024, "longitude": 1024},
                bbox=bbox,
                crs="EPSG:4326",
                resolution=img_x_res
            )
            
            if s1_avail:
                # Read the scenes into dask array structure and make persistant in memory
                signed_s1_items = [planetary_computer.sign(item) for item in s1_items]

                sen1_scns_xa = odc.stac.stac_load(
                    signed_s1_items,
                    bands=["vh"],
                    groupby="solar_day",
                    chunks={"time":12, "latitude": 1024, "longitude": 1024},
                    bbox=bbox,
                    crs="EPSG:4326",
                    resolution=img_x_res
                )
                sen1_scns_dB_xa = 10 * numpy.log10(sen1_scns_xa)
                sen1_scns_dB_xa["vh"] = sen1_scns_dB_xa.vh.where(sen1_scns_dB_xa.vh>-30)
                sen1_scns_dB_min_xa = sen1_scns_dB_xa.min(dim='time', skipna=True).compute()

            # Store the dataset input dask cluster memory
            # Comment out for larger datasets which don't fit into memory.
            sen2_scn_xa = sen2_scn_xa.persist()

            # Apply cloud masks etc.
            sen2_scn_xa = sen2_scn_xa.map_blocks(apply_sen2_vld_msk, kwargs={"bands":bands})

            # 'Clean' up the bands to remove any values less than zero - shouldn't be needed but just incase...
            sen2_scn_xa['B03'] = sen2_scn_xa.B03.where(sen2_scn_xa.B03>0)
            sen2_scn_xa['B04'] = sen2_scn_xa.B04.where(sen2_scn_xa.B04>0)
            sen2_scn_xa['B08'] = sen2_scn_xa.B08.where(sen2_scn_xa.B08>0)
            sen2_scn_xa['B11'] = sen2_scn_xa.B11.where(sen2_scn_xa.B11>0)

            # Calculate the NDWI (two versions)
            ndwi1_scn_xa = ((sen2_scn_xa.B03-sen2_scn_xa.B08)/(sen2_scn_xa.B03+sen2_scn_xa.B08))
            ndwi2_scn_xa = ((sen2_scn_xa.B03-sen2_scn_xa.B11)/(sen2_scn_xa.B03+sen2_scn_xa.B11))
            
            # Calculate the NDVI
            ndvi_scn_xa = ((sen2_scn_xa.B08-sen2_scn_xa.B04)/(sen2_scn_xa.B08+sen2_scn_xa.B04))
            
            # Calculate the MVI
            mvi_scn_xa = (sen2_scn_xa.B08-sen2_scn_xa.B03)/(sen2_scn_xa.B11+sen2_scn_xa.B03)

            # Apply threshold
            water1_pxls_xa = ndwi1_scn_xa > -0.1
            water2_pxls_xa = ndwi2_scn_xa > 0.15
            veg_pxls_xa = ndvi_scn_xa < 0.1
            mng_pxls_xa = mvi_scn_xa < 0.1

            # Summarise the changes by summing through time.
            water1_pxls_count_xa = water1_pxls_xa.sum(dim="time", skipna=True)
            water2_pxls_count_xa = water2_pxls_xa.sum(dim="time", skipna=True)
            veg_pxls_count_xa = veg_pxls_xa.sum(dim="time", skipna=True)
            mng_pxls_count_xa = mng_pxls_xa.sum(dim="time", skipna=True)
            
            dask.compute(water1_pxls_count_xa, water2_pxls_count_xa, veg_pxls_count_xa, mng_pxls_count_xa)
            
            # Update the GMW extent.
            gmw_ext_msk[water1_pxls_count_xa.values > 4] = 0
            gmw_ext_msk[water2_pxls_count_xa.values > 4] = 0
            gmw_ext_msk[veg_pxls_count_xa.values > 4] = 0
            gmw_ext_msk[mng_pxls_count_xa.values > 4] = 0
            if s1_avail:
                gmw_ext_msk[sen1_scns_dB_min_xa["vh"].values < -19] = 0
            
            # Apply erosion to resulting mask.
            gmw_ext_msk_erode = ndimage.binary_erosion(gmw_ext_msk, structure=morph_op5)

            # Get the image shape (i.e., number of pixels)
            img_shp = gmw_ext_msk.shape
            
            # Define the output image spatial transformation.
            out_img_transform = rasterio.transform.Affine(img_x_res, 0.0, bbox[0], 0.0, img_y_res, bbox[3])
           
            with rasterio.open(out_img_file,
                                'w',
                                driver='COG',
                                height=img_shp[0],
                                width=img_shp[1],
                                count=2,
                                dtype=numpy.uint8,
                                crs='epsg:4326',
                                transform=out_img_transform,
                            ) as out_img_dataset:
            
                # Write output array to the image file
                out_img_dataset.write(gmw_ext_msk, 1)
                out_img_dataset.set_band_description(1, "MngExt2018")
                
                out_img_dataset.write(gmw_ext_msk_erode, 2)
                out_img_dataset.set_band_description(2, "MngExt2018Erode")


            # Delete the data arrays to as not used any more.
            del sen2_scn_xa
            del ndwi1_scn_xa
            del ndwi2_scn_xa
            del ndvi_scn_xa
            del mvi_scn_xa
            del water1_pxls_count_xa
            del water2_pxls_count_xa
            del veg_pxls_count_xa
            del mng_pxls_count_xa
            del gmw_ext_msk
            del gmw_ext_msk_erode
            if s1_avail:
                del sen1_scns_xa
                del sen1_scns_dB_xa
                del sen1_scns_dB_min_xa
            # Restart the dask workers to ensure all the memory etc. is cleared.
            #client.restart(wait_for_workers=False)
    # Increment the tile number for user feedback.
    n_tile += 1
    

In [ ]:
# Close the dask cluster
client.close()
cluster.close()